In [ ]:
%pip install beautifulsoup4
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import os


def clean_salary_job(val):
    if pd.isna(val):
        return np.nan
    
    val = str(val).strip()
    
    if val.isnumeric():
        return int(val)
    
    if val.endswith("L"):
        val = val.replace("L", "")
        if val.replace('.', '', 1).isdigit():
            return float(val) * 100000
    
    if val.endswith("k"):
        val = val.replace("k", "")
        if val.replace('.', '', 1).isdigit():
            return float(val) * 1000
    
    try:
        return float(val)
    except:
        return np.nan


def scrape_city_data(city, headers, max_pages=11):
    """Scrape company data for a specific city"""
    url_for_city = f"https://www.ambitionbox.com/companies-in-{city}"
    dictionary = {
        "company_name": [],
        "rating": [],
        "bio": [],
        "salary": [],
        "job": [],
        "founded_in": []
    }
    
    for page in range(1, max_pages + 1):
        print(f"Scraping {city} - Page {page}")
        
        try:
            pages_url = f"{url_for_city}?page={page}"
            resp = requests.get(pages_url, headers=headers, timeout=10)
            resp.raise_for_status()  
            
            soup = BeautifulSoup(resp.content, 'html.parser')
            company_card_wrappers = soup.find_all("div", class_="companyCardWrapper")
            
            if not company_card_wrappers:
                print(f"No company cards found on page {page} for {city}")
                break
            
            for data in company_card_wrappers:
                company_name_elem = data.find("h2")
                company_name = company_name_elem.text.strip() if company_name_elem else np.nan
                dictionary["company_name"].append(company_name)
                
               
                rating_elem = data.find("div", class_="rating_text")
                rating = rating_elem.text.strip() if rating_elem else np.nan
                dictionary["rating"].append(rating)
                
                
                bio_elem = data.find("span", class_="companyCardWrapper__interLinking")
                bio = bio_elem.text.strip() if bio_elem else np.nan
                dictionary["bio"].append(bio)
                
            
                if company_name and company_name != np.nan:
                    company_slug = company_name.lower().replace(' ', '-').replace('.', '').replace(',', '')
                    overview_url = f"https://www.ambitionbox.com/overview/{company_slug}-overview"
                    
                    try:
                        time.sleep(1)  
                        resp1 = requests.get(overview_url, headers=headers, timeout=10)
                        
                        if resp1.status_code == 200:
                            new_soup = BeautifulSoup(resp1.content, 'html.parser')
                            
                            salary = np.nan
                            a_tag_salary = new_soup.find("a", title=lambda x: x and "Salaries" in x)
                            if a_tag_salary:
                                salary_elem = a_tag_salary.find("div", class_="text-primary-text font-pn-600 text-xs")
                                if salary_elem:
                                    salary = salary_elem.text.strip()
                            dictionary["salary"].append(salary)
                            
                        
                            job = np.nan
                            a_tag_job = new_soup.find("a", title=lambda x: x and "Jobs" in x)
                            if a_tag_job:
                                job_elem = a_tag_job.find("div", class_="text-primary-text font-pn-600 text-xs")
                                if job_elem:
                                    job = job_elem.text.strip()
                            dictionary["job"].append(job)
                            
                        
                            founded_year = np.nan
                            year_elements = new_soup.find_all("div", 
                                class_="inline whitespace-pre-wrap break-words text-primary-text text-sm font-pn-600 flex-[6] md:flex-[auto]")
                            if year_elements and len(year_elements) > 0:
                                founded_year = year_elements[0].text.strip()
                            dictionary["founded_in"].append(founded_year)
                            
                        else:
                           
                            dictionary["salary"].append(np.nan)
                            dictionary["job"].append(np.nan)
                            dictionary["founded_in"].append(np.nan)
                            
                    except Exception as e:
                        print(f"Error scraping {company_name}: {e}")
                        dictionary["salary"].append(np.nan)
                        dictionary["job"].append(np.nan)
                        dictionary["founded_in"].append(np.nan)
                else:
                    dictionary["salary"].append(np.nan)
                    dictionary["job"].append(np.nan)
                    dictionary["founded_in"].append(np.nan)
            
            time.sleep(2)  
        
        except requests.RequestException as e:
            print(f"Error accessing page {page} for {city}: {e}")
            break
    
    return pd.DataFrame(dictionary)


def main():
    cities = ["new-delhi", "mumbai", "pune", "bangalore", "chennai", "noida", "jaipur", "surat"]
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Cache-Control": "max-age=0",
    }
    
   
    output_dir = "scraped_data"
    os.makedirs(output_dir, exist_ok=True)
    
    
    for city in cities:
        print(f"\n{'='*50}")
        print(f"Scraping data for {city}")
        print(f"{'='*50}")
        
        try:
            df = scrape_city_data(city, headers, max_pages=11)
            
            if not df.empty:
              
                raw_filename = os.path.join(output_dir, f"{city}_raw.csv")
                df.to_csv(raw_filename, index=False)
                print(f"Saved raw data for {city} with {len(df)} records")
            else:
                print(f"No data scraped for {city}")
                
        except Exception as e:
            print(f"Failed to scrape {city}: {e}")
    
    print(f"\n{'='*50}")
    print("Processing scraped data")
    print(f"{'='*50}")
    
    for city in cities:
        filepath = os.path.join(output_dir, f"{city}_raw.csv")
        
        if os.path.exists(filepath):
            try:
                df = pd.read_csv(filepath)
                
                if not df.empty:
                    df["job"] = df["job"].apply(clean_salary_job)
                    df["salary"] = df["salary"].apply(clean_salary_job)
                    
                   
                    if df["job"].notna().any():
                        median_job = df["job"].median()
                        df["job"].fillna(median_job, inplace=True)
                    
                    if df["salary"].notna().any():
                        median_salary = df["salary"].median()
                        df["salary"].fillna(median_salary, inplace=True)
                    
                   
                    df[["field", "other"]] = df["bio"].str.split("|", n=1, expand=True)
                
                   
                    df["field"] = df["field"].str.strip() if df["field"].notna().any() else df["field"]
                    df["other"] = df["other"].str.strip() if df["other"].notna().any() else df["other"]
                    
                    processed_filename = os.path.join(output_dir, f"{city}_processed.csv")
                    df.to_csv(processed_filename, index=False)
                    
                    print(f"\n{city} -------------->")
                    print(f"Total records: {len(df)}")
                    print(f"Missing in job column: {df['job'].isnull().sum()}")
                    print(f"Missing in salary column: {df['salary'].isnull().sum()}")
                    print(f"Missing in founded_in column: {df['founded_in'].isnull().sum()}")
                    
            except Exception as e:
                print(f"Error processing {city}: {e}")
        else:
            print(f"No data file found for {city}")


if __name__ == "__main__":
    main()

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\Chhaya Jangid\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



Scraping data for new-delhi
Scraping new-delhi - Page 1
Scraping new-delhi - Page 2
Scraping new-delhi - Page 3
Scraping new-delhi - Page 4
Scraping new-delhi - Page 5
Scraping new-delhi - Page 6
Scraping new-delhi - Page 7
Scraping new-delhi - Page 8
Scraping new-delhi - Page 9
Scraping new-delhi - Page 10
Scraping new-delhi - Page 11
Saved raw data for new-delhi with 220 records

Scraping data for mumbai
Scraping mumbai - Page 1
Scraping mumbai - Page 2
Scraping mumbai - Page 3
Scraping mumbai - Page 4
Scraping mumbai - Page 5
Scraping mumbai - Page 6
Scraping mumbai - Page 7
Scraping mumbai - Page 8
Scraping mumbai - Page 9
Scraping mumbai - Page 10
Scraping mumbai - Page 11
Saved raw data for mumbai with 220 records

Scraping data for pune
Scraping pune - Page 1
Scraping pune - Page 2
Scraping pune - Page 3
Scraping pune - Page 4
Scraping pune - Page 5
Scraping pune - Page 6
Scraping pune - Page 7
Scraping pune - Page 8
Scraping pune - Page 9
Scraping pune - Page 10
Scraping pune 

C:\Users\Chhaya Jangid\AppData\Local\Temp\ipykernel_20384\3157148008.py:200: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["job"].fillna(median_job, inplace=True)
C:\Users\Chhaya Jangid\AppData\Local\Temp\ipykernel_20384\3157148008.py:204: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a 

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import os
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.utils import get_column_letter
import warnings
warnings.filterwarnings('ignore')


def clean_salary_job(val):
    if pd.isna(val):
        return np.nan
    
    val = str(val).strip()
    
    if val.isnumeric():
        return int(val)
    
    if val.endswith("L"):
        val = val.replace("L", "")
        if val.replace('.', '', 1).isdigit():
            return float(val) * 100000
    
    if val.endswith("k"):
        val = val.replace("k", "")
        if val.replace('.', '', 1).isdigit():
            return float(val) * 1000
    
    try:
        return float(val)
    except:
        return np.nan


def format_excel_file(filepath, sheet_name='Sheet1'):
    """Apply formatting to Excel file"""
    from openpyxl import load_workbook
    
    wb = load_workbook(filepath)
    ws = wb[sheet_name]
    
    # Define styles
    header_fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
    header_font = Font(color="FFFFFF", bold=True, size=12)
    border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )
    
    # Format header row
    for cell in ws[1]:
        cell.fill = header_fill
        cell.font = header_font
        cell.alignment = Alignment(horizontal="center", vertical="center")
        cell.border = border
    
    # Format data rows
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
        for cell in row:
            cell.border = border
            cell.alignment = Alignment(horizontal="left", vertical="center")
    
    # Auto-adjust column widths
    for column in ws.columns:
        max_length = 0
        column_letter = get_column_letter(column[0].column)
        
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        
        adjusted_width = min(max_length + 4, 50)
        ws.column_dimensions[column_letter].width = adjusted_width
    
    # Format currency columns
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
        # Assuming salary is in column D (4th column)
        salary_cell = row[3] if len(row) > 3 else None
        if salary_cell and isinstance(salary_cell.value, (int, float)):
            salary_cell.number_format = '"₹"#,##0'
    
    wb.save(filepath)


def scrape_city_data(city, headers, max_pages=11):
    """Scrape company data for a specific city"""
    url_for_city = f"https://www.ambitionbox.com/companies-in-{city}"
    dictionary = {
        "company_name": [],
        "rating": [],
        "bio": [],
        "salary": [],
        "job": [],
        "founded_in": []
    }
    
    for page in range(1, max_pages + 1):
        print(f"Scraping {city} - Page {page}")
        
        try:
            pages_url = f"{url_for_city}?page={page}"
            resp = requests.get(pages_url, headers=headers, timeout=10)
            resp.raise_for_status()
            
            soup = BeautifulSoup(resp.content, 'html.parser')
            company_card_wrappers = soup.find_all("div", class_="companyCardWrapper")
            
            if not company_card_wrappers:
                print(f"No company cards found on page {page} for {city}")
                break
            
            for data in company_card_wrappers:
                # Extract company name
                company_name_elem = data.find("h2")
                company_name = company_name_elem.text.strip() if company_name_elem else np.nan
                dictionary["company_name"].append(company_name)
                
                # Extract rating
                rating_elem = data.find("div", class_="rating_text")
                rating = rating_elem.text.strip() if rating_elem else np.nan
                dictionary["rating"].append(rating)
                
                # Extract bio
                bio_elem = data.find("span", class_="companyCardWrapper__interLinking")
                bio = bio_elem.text.strip() if bio_elem else np.nan
                dictionary["bio"].append(bio)
                
                # Scrape additional details
                if company_name and company_name != np.nan:
                    company_slug = company_name.lower().replace(' ', '-').replace('.', '').replace(',', '')
                    overview_url = f"https://www.ambitionbox.com/overview/{company_slug}-overview"
                    
                    try:
                        time.sleep(1)
                        resp1 = requests.get(overview_url, headers=headers, timeout=10)
                        
                        if resp1.status_code == 200:
                            new_soup = BeautifulSoup(resp1.content, 'html.parser')
                            
                            # Extract salary
                            salary = np.nan
                            a_tag_salary = new_soup.find("a", title=lambda x: x and "Salaries" in x)
                            if a_tag_salary:
                                salary_elem = a_tag_salary.find("div", class_="text-primary-text font-pn-600 text-xs")
                                if salary_elem:
                                    salary = salary_elem.text.strip()
                            dictionary["salary"].append(salary)
                            
                            # Extract job count
                            job = np.nan
                            a_tag_job = new_soup.find("a", title=lambda x: x and "Jobs" in x)
                            if a_tag_job:
                                job_elem = a_tag_job.find("div", class_="text-primary-text font-pn-600 text-xs")
                                if job_elem:
                                    job = job_elem.text.strip()
                            dictionary["job"].append(job)
                            
                            # Extract founded year
                            founded_year = np.nan
                            year_elements = new_soup.find_all("div", 
                                class_="inline whitespace-pre-wrap break-words text-primary-text text-sm font-pn-600 flex-[6] md:flex-[auto]")
                            if year_elements and len(year_elements) > 0:
                                founded_year = year_elements[0].text.strip()
                            dictionary["founded_in"].append(founded_year)
                            
                        else:
                            dictionary["salary"].append(np.nan)
                            dictionary["job"].append(np.nan)
                            dictionary["founded_in"].append(np.nan)
                            
                    except Exception as e:
                        print(f"Error scraping {company_name}: {e}")
                        dictionary["salary"].append(np.nan)
                        dictionary["job"].append(np.nan)
                        dictionary["founded_in"].append(np.nan)
                else:
                    dictionary["salary"].append(np.nan)
                    dictionary["job"].append(np.nan)
                    dictionary["founded_in"].append(np.nan)
            
            time.sleep(2)
        
        except requests.RequestException as e:
            print(f"Error accessing page {page} for {city}: {e}")
            break
    
    return pd.DataFrame(dictionary)


def create_excel_report(city_dfs, output_dir):
    """Create comprehensive Excel report with multiple sheets"""
    import datetime
    
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    excel_file = os.path.join(output_dir, f"Company_Data_Report_{timestamp}.xlsx")
    
    with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
        # Create summary sheet
        summary_data = []
        
        for city_name, df in city_dfs.items():
            if not df.empty:
                summary_data.append({
                    'City': city_name.replace('-', ' ').title(),
                    'Total Companies': len(df),
                    'Average Rating': round(df['rating'].mean(), 2),
                    'Average Salary (₹)': round(df['salary'].mean(), 2),
                    'Total Jobs': int(df['job'].sum()),
                    'Top Rated Company': df.loc[df['rating'].idxmax(), 'company_name'][:30] if 'company_name' in df.columns else 'N/A',
                    'Highest Salary (₹)': round(df['salary'].max(), 2)
                })
                
                # Save individual city sheet
                sheet_name = city_name[:31]  # Excel sheet name max 31 chars
                df.to_excel(writer, sheet_name=sheet_name, index=False)
        
        # Create summary DataFrame
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='Summary', index=False)
        
        # Create analysis sheet with top companies
        all_data = pd.concat(city_dfs.values(), ignore_index=True)
        all_data['city'] = [city for city, df in city_dfs.items() for _ in range(len(df))]
        
        # Top 20 companies by rating
        top_20_by_rating = all_data.nlargest(20, 'rating')[['company_name', 'city', 'rating', 'salary', 'job', 'founded_in']]
        top_20_by_rating.to_excel(writer, sheet_name='Top 20 Companies', index=False)
        
        # Top 20 companies by salary
        top_20_by_salary = all_data.nlargest(20, 'salary')[['company_name', 'city', 'rating', 'salary', 'job', 'founded_in']]
        top_20_by_salary.to_excel(writer, sheet_name='Top Salaries', index=False)
    
    # Format the Excel file
    format_excel_file(excel_file, 'Summary')
    
    print(f"\n✅ Excel report created: {excel_file}")
    return excel_file


def main():
    cities = ["new-delhi", "mumbai", "pune", "bangalore", "chennai", "noida", "jaipur", "surat"]
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Cache-Control": "max-age=0",
    }
    
    # Create output directory
    output_dir = "excel_reports"
    os.makedirs(output_dir, exist_ok=True)
    
    city_dataframes = {}
    
    # Step 1: Scrape data for each city
    for city in cities:
        print(f"\n{'='*50}")
        print(f"Scraping data for {city}")
        print(f"{'='*50}")
        
        try:
            df = scrape_city_data(city, headers, max_pages=11)
            
            if not df.empty:
                # Clean and process data
                df["job"] = df["job"].apply(clean_salary_job)
                df["salary"] = df["salary"].apply(clean_salary_job)
                
                # Fill missing values with median
                if df["job"].notna().any():
                    median_job = df["job"].median()
                    df["job"].fillna(median_job, inplace=True)
                
                if df["salary"].notna().any():
                    median_salary = df["salary"].median()
                    df["salary"].fillna(median_salary, inplace=True)
                
                # Split bio column
                df[["field", "other"]] = df["bio"].str.split("|", n=1, expand=True)
                df["field"] = df["field"].str.strip() if df["field"].notna().any() else df["field"]
                df["other"] = df["other"].str.strip() if df["other"].notna().any() else df["other"]
                
                # Save individual Excel file for this city
                city_excel = os.path.join(output_dir, f"{city}_companies.xlsx")
                df.to_excel(city_excel, index=False)
                format_excel_file(city_excel)
                print(f"✅ Individual Excel file saved: {city_excel}")
                
                # Store for combined report
                city_dataframes[city] = df
                
            else:
                print(f"No data scraped for {city}")
                
        except Exception as e:
            print(f"Failed to scrape {city}: {e}")
    
    # Step 2: Create comprehensive Excel report
    if city_dataframes:
        excel_report = create_excel_report(city_dataframes, output_dir)
        
        # Display summary
        print(f"\n{'='*60}")
        print("📊 EXCEL REPORTS CREATED SUCCESSFULLY!")
        print(f"{'='*60}")
        
        for city, df in city_dataframes.items():
            print(f"\n{city.replace('-', ' ').title():15} | Companies: {len(df):4} | Avg Rating: {df['rating'].mean():.2f} | Avg Salary: ₹{df['salary'].mean():,.0f}")
        
        print(f"\n📁 All Excel files are saved in: {os.path.abspath(output_dir)}")
        print(f"📊 Main report file: {excel_report}")
        
        # Open Excel file automatically (Windows only)
        try:
            os.startfile(excel_report)
            print("📂 Excel file opened automatically!")
        except:
            print("\n📂 To open the Excel file, navigate to:")
            print(f"   {os.path.abspath(excel_report)}")
    else:
        print("\n❌ No data was scraped. Please check your internet connection or the website.")


def view_excel_data():
    """View the created Excel files"""
    import subprocess
    import glob
    
    output_dir = "excel_reports"
    
    if not os.path.exists(output_dir):
        print("❌ No Excel reports found. Please run the scraper first.")
        return
    
    excel_files = glob.glob(os.path.join(output_dir, "*.xlsx"))
    
    if not excel_files:
        print("❌ No Excel files found in the reports directory.")
        return
    
    print(f"\n{'='*60}")
    print("📁 AVAILABLE EXCEL FILES")
    print(f"{'='*60}")
    
    for i, file in enumerate(sorted(excel_files), 1):
        file_size = os.path.getsize(file) / 1024  # Size in KB
        print(f"{i:2}. {os.path.basename(file):40} ({file_size:.1f} KB)")
    
    print(f"\nTotal Excel files: {len(excel_files)}")
    
    while True:
        choice = input("\nEnter file number to open, 'a' to open all, or 'q' to quit: ").lower()
        
        if choice == 'q':
            break
        elif choice == 'a':
            for file in excel_files:
                try:
                    os.startfile(file)
                    print(f"Opened: {os.path.basename(file)}")
                except:
                    print(f"Could not open: {os.path.basename(file)}")
            break
        else:
            try:
                idx = int(choice) - 1
                if 0 <= idx < len(excel_files):
                    file_to_open = excel_files[idx]
                    try:
                        os.startfile(file_to_open)
                        print(f"✅ Opened: {os.path.basename(file_to_open)}")
                        
                        # Preview file contents
                        df = pd.read_excel(file_to_open, sheet_name=None)
                        print(f"\n📋 File contains {len(df)} sheet(s):")
                        for sheet_name, sheet_data in df.items():
                            print(f"   - {sheet_name}: {len(sheet_data)} rows × {len(sheet_data.columns)} columns")
                        
                    except Exception as e:
                        print(f"❌ Could not open file: {e}")
                else:
                    print("❌ Invalid number. Please try again.")
            except ValueError:
                print("❌ Invalid input. Please enter a number, 'a', or 'q'.")


if __name__ == "__main__":
    print("🏢 AMBITIONBOX COMPANY DATA SCRAPER")
    print("="*50)
    
    print("\nOptions:")
    print("1. Scrape data and create Excel reports")
    print("2. View existing Excel reports")
    print("3. Both (scrape first, then view)")
    
    choice = input("\nSelect option (1, 2, or 3): ").strip()
    
    if choice in ["1", "3"]:
        print("\n" + "="*60)
        print("🚀 STARTING DATA SCRAPING")
        print("="*60)
        main()
    
    if choice in ["2", "3"]:
        view_excel_data()

ModuleNotFoundError: No module named 'openpyxl'